In [5]:
import requests
from bs4 import BeautifulSoup
# 多线程执行
import threading
# 延迟执行
"""
    author:李梓瑀
    start_date:2024-03-11
    end_date:2024-03-12
    待爬取信息:同花顺表格
    难点:ip封锁,请求头Cookie特殊参数,环境缺失
    解决方法:使用本地ip池,逆向补齐环境
"""
requests.models.Response.encoding = 'utf-8'
# 请求列表,获取前12页数据
urllist = [f"https://q.10jqka.com.cn/zs/index/field/zdf/order/desc/page/{i}/ajax/1/" for i in range(1,13)]
# ip地址池列表
ipUrl = "http://localhost:5555/random"
# 存储请求头
myHeaders = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Host":"q.10jqka.com.cn",
    "Referer":"https://q.10jqka.com.cn/zs/"
}
# 设置数据列表
dataList = []
# 设置线程池
threads = []
# 创建表格类,存储每一列
class TableData:
    # 序号
    number:str = None
    # 代码
    code:str = None
    # 名称
    name:str = None
    # 最新价
    newPrice:str = None
     # 涨跌额
    upsAndDowns:str = None
    # # 涨跌幅
    quoteChange:str = None
    # 昨收
    yesPrice:str = None
    # 今开
    toDayPrice:str = None
    # 最高价
    maxPrice:str = None
    # 最底价
    minPrice:str = None
    # 成交量
    volume:str = None
    # 成交价格
    volumePrice:str = None
    # 全参构造
    def __init__(self,number,code,name,newPrice,upsAndDowns,quoteChange,yesPrice,toDayPrice,maxPrice,minPrice,volume,volumePrice):
        self.number = number
        self.code = code
        self.name = name
        self.newPrice = newPrice
        self.upsAndDowns = upsAndDowns
        self.quoteChange = quoteChange
        self.yesPrice = yesPrice
        self.toDayPrice = toDayPrice
        self.maxPrice = maxPrice
        self.minPrice = minPrice
        self.volume = volume
        self.volumePrice = volumePrice


In [6]:
import execjs
# 获取js中的cookie加密参数v
def getCK():
    text = execjs.compile(open(r'./js/同花顺cookie生成算法.js').read())
    return text.call("getCK").split(";")[0].strip("v=")

In [7]:
# 从地址池获取地址
def getIpByPools():
    ipUrl = "http://localhost:5555/random"
    res = requests.get(ipUrl)
    return res.text
# 获取表格数据存入数据列表
def getTableData(url):
    # 渲染页面
    global dataList
    proxies = {
        'http': getIpByPools().split(":")[0]
    }
    ck = {
        "v":getCK()
    }
    res = requests.get(url=url,headers=myHeaders,proxies=proxies,cookies=ck)
    bs_html = BeautifulSoup(res.text)
    for tr in bs_html.select(".m-table tbody tr"):
        # 空列表存储数据
        mylis = []
        for td in tr.select("td"):
            data = td.text.split()
            mylis.append(data)
        #  设置表格数据
        table = TableData(mylis[0][0],mylis[1][0],mylis[2][0],mylis[3][0],mylis[4][0],mylis[5][0],mylis[6][0],mylis[7][0],mylis[8][0],mylis[9][0],mylis[10][0],mylis[11][0])
        dataList.append(table)
# 多线程爬取
def getTonghua():
    global threads
    for url in urllist:
        # 开线程
        thread = threading.Thread(target=getTableData,args=(url,))
        thread.start()
        # 加入线程池
        threads.append(thread)
    for thread in threads:
        thread.join()
    print("线程结束")
getTonghua()

线程结束


In [8]:
# 存储数据
import openpyxl
import os
import datetime
import time as times
# 使用openpyxl存储数据
# 新建工作簿
path = r"../数据集/同花顺大A.xlsx"
# 没有则新建,有则获取到sheet工作表
titleHeader = ["序号","指数代码","指数名称","最新价","涨跌额","涨跌幅%","昨收","今开","最高价","最低价","成交量(万手)","成交额(亿元)"]
# 如果有则创建
if not os.path.exists(path):
    wb = openpyxl.Workbook()
    wb.save(path)
else:
    wb = openpyxl.load_workbook(path)
# 移除默认工作表
if "Sheet" in wb.sheetnames:
    wb.remove(wb['Sheet'])
# 创建与今天日期相同的sheet工作表
date = datetime.datetime.now()
year = date.year
month = date.month
day = date.day
time = f"{year}_{month}_{day}"
if time not in wb.sheetnames:
    wb.create_sheet(time)
# 存储数据
wb.save(path)
sheet = wb[time]
# 设置单元格标题
if sheet.cell(1,1).value != titleHeader[0]:
    for i in range(1,len(titleHeader)+1):
        sheet.cell(1,i).value = titleHeader[i-1]
    wb.save(path)
# 存入数据
for data in dataList:
    # 对象解构为列表
   sheet.append(list(vars(data).values()))
# 设置一下列宽
sheet.column_dimensions['K'].width = 30
sheet.column_dimensions['L'].width = 30
wb.save(path)
times.sleep(5)
print("爬虫完成!")

爬虫完成!
